In [1]:
import pandas as pd
import glob
import os
import time
from tqdm import tqdm

# Step 2 - degrade frameRate

We degrade the data to be fps=5 instead of fps=25.

### Explore merged_tracks_tracksMeta

In [2]:
folder_path = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\01_merged_tracks_tracksMeta_data'
file_name_merged_tracks_tracksMeta_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
file_name_merged_tracks_tracksMeta_list

['18_merged_tracks_tracksMeta.csv',
 '19_merged_tracks_tracksMeta.csv',
 '20_merged_tracks_tracksMeta.csv',
 '21_merged_tracks_tracksMeta.csv',
 '22_merged_tracks_tracksMeta.csv',
 '23_merged_tracks_tracksMeta.csv',
 '24_merged_tracks_tracksMeta.csv',
 '25_merged_tracks_tracksMeta.csv',
 '26_merged_tracks_tracksMeta.csv',
 '27_merged_tracks_tracksMeta.csv',
 '28_merged_tracks_tracksMeta.csv',
 '29_merged_tracks_tracksMeta.csv']

In [3]:
# print dataframes info and statastics
def print_df_stats(file_name_merged_tracks_tracksMeta_list, folder_path):
    # go to merged data folder
    os.chdir(folder_path)
    
    for i in range(len(file_name_merged_tracks_tracksMeta_list)):
        print('-------- ',file_name_merged_tracks_tracksMeta_list[i], ":",' --------','\n')
        df=pd.read_csv(file_name_merged_tracks_tracksMeta_list[i])
        # print info
        print(df.info())
        print()
        
        # count how much tracks for each class
        print('count classes:')
        # print(df['class'].value_counts(), '\n')
        unique_trackId_counts = df.groupby('class')['trackId'].nunique()
        print(unique_trackId_counts, '\n')
    
    # change back to parent directory
    os.chdir('..')
 
print_df_stats(file_name_merged_tracks_tracksMeta_list, folder_path)

--------  18_merged_tracks_tracksMeta.csv :  -------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625989 entries, 0 to 625988
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   recordingId    625989 non-null  int64  
 1   trackId        625989 non-null  int64  
 2   frame          625989 non-null  int64  
 3   trackLifetime  625989 non-null  int64  
 4   xCenter        625989 non-null  float64
 5   yCenter        625989 non-null  float64
 6   heading        625989 non-null  float64
 7   width          625989 non-null  float64
 8   length         625989 non-null  float64
 9   xVelocity      625989 non-null  float64
 10  yVelocity      625989 non-null  float64
 11  initialFrame   625989 non-null  int64  
 12  finalFrame     625989 non-null  int64  
 13  numFrames      625989 non-null  int64  
 14  class          625989 non-null  object 
dtypes: float64(7), int64(7), object(1)
memory usage: 71.6+ MB
None


In [4]:
os.getcwd() # show current directory

'C:\\Users\\yftac\\Documents\\00_Project_Afeka\\ttc_calculation'

## Load merged data to df

In [5]:
# print dataframes info and statastics
def load_merged_dfs(file_name_merged_tracks_tracksMeta_list, folder_path):
    # go to merged data folder
    # folder_name='merged_tracks_tracksMeta_data'
    # change to working dir
    os.chdir(folder_path)
    dfs = []
    for i in range(len(file_name_merged_tracks_tracksMeta_list)):
        print(i)
        df=pd.read_csv(file_name_merged_tracks_tracksMeta_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    print('---- dfs ready ----')
    return dfs

dfs=load_merged_dfs(file_name_merged_tracks_tracksMeta_list, folder_path)

0
1
2
3
4
5
6
7
8
9
10
11
---- dfs ready ----


### degrade fps

In [6]:
def degrade_fps(df, df_name, fps_new):
    print(df_name)
    print('frame min=',df['frame'].min(), '\nframe max=',df['frame'].max())
    # iterate on all frames of df
    row_indexes_of_frame_list=[]
    new_indexes_list=[]
    for f in range(df['frame'].min(), df['frame'].max()+1, 1):
        if f%fps_new==0: # find each 5th frame number
            # find rows of frames
            row_indexes_of_frame_list = df.index[df['frame'] == f].tolist()
            # print(row_indexes_of_frame_list)
            new_indexes_list.extend(row_indexes_of_frame_list)
 
    # create a new df from the new rows indexes
    new_df = df.loc[new_indexes_list].reset_index(drop=True) 
    # divide the frame and life time to create a corrected frames and time-life column
    new_df['frame'] = new_df['frame'] // fps_new 
    new_df['trackLifetime'] = new_df['trackLifetime'] // fps_new
    print('new length of df:', len(new_df))
    print('--------------------------------------')
    return new_df

fps_new=5 # degrade factor to divide the frames

degraded_dfs=[]
# iterate all dfs
for x in range(len(dfs)):
    #degrade
    degraded_df = degrade_fps(dfs[x], file_name_merged_tracks_tracksMeta_list[x], fps_new)
    #sort by trackId and frame
    degraded_df_sorted = degraded_df.sort_values(by=['trackId', 'frame']).reset_index(drop=True)
    # drop irrelevant frame related columns:  initialFrame	finalFrame	numFrames
    degraded_df_sorted.drop(['initialFrame', 'finalFrame', 'numFrames'], axis=1, inplace=True)
    #append to dfs list
    degraded_dfs.append(degraded_df_sorted)
    
print('~~~ Finished degrading fps dataframes ~~~')

18_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 27990
new length of df: 125216
--------------------------------------
19_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 24835
new length of df: 98425
--------------------------------------
20_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 32542
new length of df: 126982
--------------------------------------
21_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 31106
new length of df: 159247
--------------------------------------
22_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 30790
new length of df: 150767
--------------------------------------
23_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 31597
new length of df: 156565
--------------------------------------
24_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 23872
new length of df: 116171
--------------------------------------
25_merged_tracks_tracksMeta.csv
frame min= 0 
frame max= 31194
new length of df: 134507
------------------------

In [7]:
### save to csv files  ###
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"
os.chdir(root_dir)

def save_dfs_degraded(degraded_dfs, recordingId_list):
    # create folder to save files 
    folder_name='02_fps_degraded_data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    # change to working dir
    os.chdir(folder_name)
    # initialize a list of the files to return by the function
    file_name_degraded_data_list=[]
    # iterate dfs for each df and save
    for x in tqdm(range(len(degraded_dfs)), desc="Saving CSV files"):
        # save as CSV file
        file_name=str(recordingId_list[x]) + '_fps_degraded_data' + '.csv'
        degraded_dfs[x].to_csv(file_name, index=False)
        # append the file name to the list
        file_name_degraded_data_list.append(file_name)
        
    current_dir = os.getcwd()
    print(f"files saved to: {current_dir}")
    # change back to parent directory
    os.chdir('..')

    return file_name_degraded_data_list

#save with recording prefix
recordingId_list=[f.split('_')[0] for f in file_name_merged_tracks_tracksMeta_list]
file_name_degraded_data_list=save_dfs_degraded(degraded_dfs, recordingId_list)

Saving CSV files: 100%|██████████| 12/12 [00:08<00:00,  1.46it/s]

files saved to: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\02_fps_degraded_data


In [8]:
file_name_degraded_data_list

['18_fps_degraded_data.csv',
 '19_fps_degraded_data.csv',
 '20_fps_degraded_data.csv',
 '21_fps_degraded_data.csv',
 '22_fps_degraded_data.csv',
 '23_fps_degraded_data.csv',
 '24_fps_degraded_data.csv',
 '25_fps_degraded_data.csv',
 '26_fps_degraded_data.csv',
 '27_fps_degraded_data.csv',
 '28_fps_degraded_data.csv',
 '29_fps_degraded_data.csv']